In [239]:
import csv
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def lesk(word, sentence):
    """
    Lesk's algoritm implementation. Given a word and a sentence in which it appears,
    it returns the best sense of the word.

    :param word: word to disabiguate
    :param sentence: sentence to compare
    :return: best sense of word
    """

    # Calculating the synset of the given word inside WN
    word_senses = wn.synsets(word)
    best_sense = word_senses[0]
    max_overlap = 0

    # I choose the bag of words approach
    context = bag_of_word(sentence)

    for sense in word_senses:
        # set of words in the gloss
        signature = bag_of_word(sense.definition())

        # and examples of the given sense
        examples = sense.examples()
        for ex in examples:
            # after this line, signature will contain for all the words, their
            # bag of words definition and their examples
            signature = signature.union(bag_of_word(ex))

        overlap = compute_overlap(signature, context)
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense

    return best_sense


def bag_of_word(sent):
    """
    Auxiliary function for the Lesk algorithm. Transforms the given sentence
    according to the bag of words approach, apply lemmatization, stop words
    and punctuation removal.

    :param sent: sentence
    :return: bag of words
    """

    stop_words = set(stopwords.words('english'))
    punct = {',', ';', '(', ')', '{', '}', ':', '?', '!'}
    wnl = nltk.WordNetLemmatizer()
    tokens = nltk.word_tokenize(sent)
    tokens = list(filter(lambda x: x not in stop_words and x not in punct, tokens))
    return set(wnl.lemmatize(t) for t in tokens)


def compute_overlap(signature, context):
    """
    Auxiliary function for the Lesk algorithm. Computes the number of words in
    common between signature and context.

    :param signature: bag of words of the signature (e.g. definitions + examples)
    :param context: bag of words of the context (e.g. sentence)
    :return: number of elements in commons
    """

    return len(signature & context)


In [ ]:
# IPERONIMI
sentence = "the concept of fairness, equality for all the parts involved"

syns = wn.synsets("apple")
names = []

for i, s in enumerate(syns, start=0):
    hyper = lambda s: s.hypernyms()  # SOPRA-NOME, categoria superiore della parola
    temp = list(s.closure(hyper, depth=1))
    names.extend([x.name().split(".")[0] for x in temp])
    #print("SYN: {} \t HYPER: {}".format(s,t))

print(names)
#print(len(names))
#print(set(names))
#print(len(set(names)))

In [171]:
# IPONIMI
syns = wn.synsets("concept")
names = []

for i, s in enumerate(syns, start=0):
    hyper = lambda s: s.hyponyms()  # SOTTONOME, significato semantico incluso in altra parola
    temp = list(s.closure(hyper, depth=1))
    names.extend([x.name().split(".")[0] for x in temp])
    #print("SYN: {} \t HYPER: {}".format(s,t))

print(names)
#print(len(names))
#print(set(names))
#print(len(set(names)))

['abstraction', 'category', 'conceptualization', 'fact', 'hypothesis', 'law', 'law', 'lexicalized_concept', 'notion', 'part', 'property', 'quantity', 'rule', 'rule', 'whole']


In [ ]:
wn.synset("apple.n.01").definition()

In [ ]:
h = wn.synset("apple.n.01").hypernyms()
for v in h:
    print(v.name())

In [ ]:
wn.synset("pome.n.01").definition()

In [123]:
def load_data():
    """
    It reads che definition's CSV
    :return: four list containing the read definitions.
    """
    with open(options["output"] + 'content-to-form.csv', "r", encoding="utf-8") as content:
        cnt = csv.reader(content, delimiter=';')

        dictionary = {}
        i = 0
        for line in cnt:
            dictionary[i] = line
            i += 1

        return dictionary


def preprocess(definition):
    """
    It does some preprocess: removes stopwords, punctuation and does the
    lemmatization of the tokens inside the sentence.
    :param definition: a string representing a definition
    :return: a set of string which contains the preprocessed string tokens.
    """

    # Removing stopwords
    definition = definition.lower()
    stop_words = set(stopwords.words('english'))
    punct = {',', ';', '(', ')', '{', '}', ':', '?', '!', '.'}
    wnl = nltk.WordNetLemmatizer()
    tokens = nltk.word_tokenize(definition)
    tokens = list(filter(lambda x: x not in stop_words and x not in punct, tokens))

    # Lemmatization
    lemmatized_tokens = set(wnl.lemmatize(t) for t in tokens)

    return lemmatized_tokens


def preprocess_synset(synset):
    """
    It does some preprocess: removes the stopword, punctuation and does the
    lemmatization of the tokens inside the sentence.
    :param definition: a string representing a definition
    :return: a set of string which contains the preprocessed string tokens.
    """
    pre_synset = synset.split(".")
    clean_synset = pre_synset[0]
    return clean_synset

In [282]:
options = {
        "output": "/Users/lorenzotabasso/Desktop/University/TLN/Progetto/19-20/tln-1920/part3/exercise2/input/",
    }

content = load_data()  # Loading the content-to-form.csv file

'''
1. prendo definzione, disambiguo con pos-tagging. il primo nome è il genus
2. come approccio personalizzato, teveno un dizionario di genus (dopo aver esplorato tutte le definizioni) e espandevo solo il genus più frequente
riducendo la ricerca
3. prendo da wordnet i synsets di quel sostantivo, e per ognuno di essi parto in basso con gli iponimi
4. calcolo l'iponimo dell'iponimo dell'iponimo..., per non sclerare utilizza la closure (chiusura trasitiva). Calcolo gli iponimi fino a un certo 
livello
5. calcola iponimo con più overlapping, stili classifica
'''

for index in content:
#for index in range(1):
    
    hyponyms_list = []
    
    for definition in content[index]:
    #for definition in content[0]:
        local_genus = {}
        hyponyms = []
            
        def_tokens = word_tokenize(definition)
        results = nltk.pos_tag(def_tokens)
        
        possibles_genus = list(filter(lambda x: x[1] == "NN", results))
        # Es.: [('abstract', 'NN'), ('concept', 'NN'), ('idea', 'NN'), ('fairness', 'NN'), ('front', 'NN'), ('code', 'NN'), ('community', 'NN')]

        for g in possibles_genus:
            if not g[0] in local_genus:
                local_genus[g[0]] = 1
            else:
                local_genus[g[0]] += 1
    
#         print(index, genus)
#         print("{} - {}\n".format(index, local_genus))
        
        if len(local_genus) > 0:
            genus = max(local_genus, key=local_genus.get)
#             print("GENUS: " + genus)
        
            syns = wn.synsets(genus)
        
            # Prendiamo tutti gli iponimi per il genus della singola definizione
            for i, s in enumerate(syns, start=0):
                hypon = lambda s: s.hyponyms()  # SOTTONOME, significato semantico incluso in altra parola
                all_hypon = list(s.closure(hypon, depth=10))  # TODO: aumentare a 2,3
                hyponyms.extend([x.name().split(".")[0] for x in all_hypon])
#                 print("SYN: {} \t HYPER: {}".format(s,t))

#             print(index, hyponyms, "\n")
#         else:
#             print("NADA")
            
        
        hyponyms_list.append(' '.join(hyponyms))
    
#     print(hyponyms_list)

        
    '''
    CountVectorizer will create k vectors in n-dimensional space, where:
    - k is the number of sentences,
    - n is the number of unique words in all sentences combined.
    If a sentence contains a certain word, the value will be 1 and 0 otherwise
    '''
    
    vectorizer = CountVectorizer()
    matrix = vectorizer.fit_transform(hyponyms_list)
    
    feature_list = vectorizer.get_feature_names()
    vectors = matrix.toarray()
    
    m = vectors.sum(axis=0).argmax()
    
    print(m)
    print(feature_list[m] + '\n')
#     print(feature_list)
    

41213
s_law

937
logic

1640
wish

42557
self

7326
spot

11448
in



KeyboardInterrupt: 